In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf, re, time, math
from wmt_data import *
from seq2seq_model import *
from tensorflow.python.framework import ops
from compute_bleu import *

buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]
V_en = 50000; V_fr = 70000; max_train_data_size = 0
learning_rate = 0.5; num_samples = 512
state_size = 512; num_layers = 3

en_train, fr_train, en_dev, fr_dev, en_vocab_path, fr_vocab_path = prepare_wmt_data(V_en, V_fr)
dev_set = read_data(en_dev, fr_dev, buckets)
train_set = read_data(en_train, fr_train, buckets, max_train_data_size)

def create_model():
    ops.reset_default_graph()
    return Seq2SeqModel(V_en, V_fr, buckets, state_size, num_layers, 5.0, learning_rate, num_samples)

  reading data line 100000
  reading data line 200000
  reading data line 300000
  reading data line 400000
  reading data line 500000
  reading data line 600000
  reading data line 700000
  reading data line 800000
  reading data line 900000
  reading data line 1000000
  reading data line 1100000
  reading data line 1200000
  reading data line 1300000
  reading data line 1400000
  reading data line 1500000
  reading data line 1600000
  reading data line 1700000
  reading data line 1800000
  reading data line 1900000
  reading data line 2000000
  reading data line 2100000
  reading data line 2200000
  reading data line 2300000
  reading data line 2400000
  reading data line 2500000
  reading data line 2600000
  reading data line 2700000
  reading data line 2800000
  reading data line 2900000
  reading data line 3000000
  reading data line 3100000
  reading data line 3200000
  reading data line 3300000
  reading data line 3400000
  reading data line 3500000
  reading data line 3600000
 

In [56]:
model = create_model()
print "Loaded model"

for v in tf.all_variables():
    print v.name, v.get_shape()

Loaded model
Variable:0 ()
Variable_1:0 ()
proj_w:0 (70000, 512)
proj_b:0 (70000,)
seq2seq/encoding/embedding_en:0 (50000, 512)
seq2seq/encoding/RNN/BasicLSTMCell/Linear/Matrix:0 (1024, 2048)
seq2seq/encoding/RNN/BasicLSTMCell/Linear/Bias:0 (2048,)
seq2seq/decoder/embedding_de:0 (70000, 512)
seq2seq/decoder/w_a:0 (512, 512)
seq2seq/decoder/w_c:0 (1024, 512)
seq2seq/decoder/BasicLSTMCell/Linear/Matrix:0 (1024, 2048)
seq2seq/decoder/BasicLSTMCell/Linear/Bias:0 (2048,)


In [ ]:
t = time.time()
# model = create_model()
print "Loaded model", (time.time() - t)
saver = tf.train.Saver(tf.all_variables(), max_to_keep=2)

vocab_en, rev_vocab_en = initialize_vocabulary(en_vocab_path)
vocab_fr, rev_vocab_fr = initialize_vocabulary(fr_vocab_path)

def train(name, sampled=True, steps=50000):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver.restore(sess, "wmt/my_full_attns.ckpt-300000")
        steps_per_checkpoint = 250; batch_size = 128

        train_bucket_sizes = [len(train_set[b]) for b in xrange(len(buckets))]
        train_total_size = float(sum(train_bucket_sizes))

        train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size for i in xrange(len(train_bucket_sizes))]

        # This is the training loop.
        step_time, loss = 0.0, 0.0; global_step = 0;
        previous_losses = []
        for current_step in range(steps):

            random_number_01 = np.random.random_sample() # Smart
            bucket_id = min([i for i in xrange(len(train_buckets_scale)) if train_buckets_scale[i] > random_number_01])

            start_time = time.time()
            encoder_inputs, decoder_inputs, target_weights = get_batch(train_set, batch_size, buckets, bucket_id)
            step_loss, _ = model.train_step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, sampled)
            step_time += (time.time() - start_time) / steps_per_checkpoint
            loss += step_loss / steps_per_checkpoint
            if current_step % steps_per_checkpoint == (-1 % steps_per_checkpoint):
                perplexity = math.exp(float(loss)) if loss < 300 else float("inf")
                global_step = model.global_step.eval()
                if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
                    sess.run(model.learning_rate_decay_op)
                previous_losses.append(loss)
                bleus = computeBleu(sess, model, dev_set, rev_vocab_en, rev_vocab_fr)
                f = open('bleus_'+name+'.txt','a'); f.write(str(global_step)+' '+str(bleus)+'\n'); f.close();
                print ("global step %d learning rate %.4f step-time %.2f perplexity %.2f bleu %.2f" % (global_step, model.learning_rate.eval(), step_time, perplexity, bleus))
                step_time, loss = 0.0, 0.0
            if current_step % 1000 == (-1 % 1000):
                t = time.time()
                saver.save(sess, "wmt/"+name+".ckpt", global_step=model.global_step)
                print "Saving takes", time.time()-t, "seconds"
# train("attns", True, 200001)
train("w", False, 300000)
# train("full", False, 23000)

In [38]:
for v in tf.all_variables():
    print v.name, v.get_shape()

Variable:0 ()
Variable_1:0 ()
proj_w:0 (70000, 512)
proj_b:0 (70000,)
seq2seq/embedding_en:0 (50000, 512)
seq2seq/encoding/RNN/BasicLSTMCell/Linear/Matrix:0 (1024, 2048)
seq2seq/encoding/RNN/BasicLSTMCell/Linear/Bias:0 (2048,)
seq2seq/decoder/embedding_de:0 (50000, 512)
seq2seq/decoder/BasicLSTMCell/Linear/Matrix:0 (1024, 2048)
seq2seq/decoder/BasicLSTMCell/Linear/Bias:0 (2048,)


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
def readSSV(fn):
    f = open(fn)
    dic = {};
    for line in iter(f):
        attrs = line.split(' ')
        div[int(attrs[0])] = float(attrs[1])
    f.close()
    return dic

full_run = readSSV('sampled_bleus.txt')
sampled_run = readSSV('sampled_bleus.txt')

plt.plot(full_run.keys(), full_run.values(), label="Full Softmax")
plt.plot(sampled_run.keys(), sampled_run.values(), label="Sampled Softmax")
plt.legend(loc='upper left')
plt.xlabel("Iterations")
plt.ylabel("BLEU Score")

In [12]:
from tensorflow.python.ops import variable_scope
# model = create_model()
saver = tf.train.Saver(tf.all_variables())

with tf.Session() as sess:
    saver.restore(sess, "wmt/translate.ckpt-176800")
    train_writer = tf.train.SummaryWriter('/tensorboard', sess.graph)
    with tf.variable_scope("embedding_rnn_seq2seq"):
        with tf.variable_scope("RNN"):
            with tf.variable_scope("EmbeddingWrapper", reuse=True):
                embedding = variable_scope.get_variable("embedding", [shape])
                print embedding
                emb = np.array(embedding)
                print emb.shape
                f = open("wmt/embedding.txt", 'w')
    

NotFoundError: Tensor name "embedding" not found in checkpoint files wmt/translate.ckpt-176800
	 [[Node: save_6/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_6/Const_0, save_6/RestoreV2_2/tensor_names, save_6/RestoreV2_2/shape_and_slices)]]
	 [[Node: save_6/RestoreV2_10/_15 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_160_save_6/RestoreV2_10", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save_6/RestoreV2_2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr

In [4]:
V_en = 50000; V_fr = 70000
vocab_file = "wmt/vocab"+str(V_fr)+".fr"
data_file = "wmt/giga-fren.release2.fixed.fr"
target_path = "wmt/giga-fren.release2.fixed.ids"+str(V_fr)+".fr"

dev_data = "wmt/dev/newstest2013-ref.fr.sgm"
dev_target = "wmt/dev/newstest2013.ids"+str(V_fr)+".fr"
# create_vocabulary(vocab_file, data_file, V_fr)
data_to_token_ids(data_file, target_path, vocab_file, tokenizer=None, normalize_digits=True)
data_to_token_ids(dev_data, dev_target, vocab_file, tokenizer=None, normalize_digits=True)